##Linear Regression using TensorFlow

In this session, we will have a look at creating a linear regression model using tensorflow 2.0. Note that we already know the basics of linear regression and understand the implementation through sklearn. We will try to figure out how to do it using tensorflow tools that we have learnt.   


<i>Note that to keep this exercise simple and focused on tensorflow and its relevant functions, we will make a very simple model with very basic preprocessing.</i> 

#### Let us start with mounting our drive to be able to use the dataset stored in our dataset.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

ModuleNotFoundError: No module named 'google.colab'

#### Now, let us install the latest version of tensorflow.


In [2]:
!pip install tensorflow==2.0.0

     |███████████████████████████▍    | 73.9MB 1.2MB/s eta 0:00:10
ERROR: Operation cancelled by user


##### Check the version of the installation

In [3]:
import tensorflow as tf
print(tf.__version__)

2.2.0


Now Let us import out data and get it ready for modelling.

In [4]:
import pandas as pd
import numpy as np


cars_data = pd.read_csv('/content/drive/My Drive/Colab Notebooks/usedcars.csv')
cars_data

,year,model,price,mileage,color,transmission
0,2011,SEL,21992,7413,Yellow,AUTO
1,2011,SEL,20995,10926,Gray,AUTO
2,2011,SEL,19995,7351,Silver,AUTO
3,2011,SEL,17809,11613,Gray,AUTO
4,2012,SE,17500,8367,White,AUTO
...,...,...,...,...,...,...
145,2006,SES,6200,95000,Silver,AUTO
146,2002,SE,5995,87003,Red,AUTO
147,2000,SE,5980,96841,Red,AUTO
148,2001,SE,4899,151479,Yellow,AUTO


In [5]:
cars_data.isna().sum()

year            0
model           0
price           0
mileage         0
color           0
transmission    0
dtype: int64

In [6]:
#just some basic preprocessing
cars_data.drop_duplicates()
#creating dummy variables for the categorical features
cars_data = pd.get_dummies(cars_data)
cars_data = cars_data.astype('float32') # we will need to convert the dataset to float in order to be able to convert it into tensors later.
cars_data


,year,price,mileage,model_SE,model_SEL,model_SES,color_Black,color_Blue,color_Gold,color_Gray,color_Green,color_Red,color_Silver,color_White,color_Yellow,transmission_AUTO,transmission_MANUAL
0,2011.0,21992.0,7413.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
1,2011.0,20995.0,10926.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,2011.0,19995.0,7351.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
3,2011.0,17809.0,11613.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,2012.0,17500.0,8367.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145,2006.0,6200.0,95000.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
146,2002.0,5995.0,87003.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
147,2000.0,5980.0,96841.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
148,2001.0,4899.0,151479.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0


In [7]:
#explorinfg column names
cars_data.columns

Index(['year', 'price', 'mileage', 'model_SE', 'model_SEL', 'model_SES',
       'color_Black', 'color_Blue', 'color_Gold', 'color_Gray', 'color_Green',
       'color_Red', 'color_Silver', 'color_White', 'color_Yellow',
       'transmission_AUTO', 'transmission_MANUAL'],
      dtype='object')

In [10]:
#getting the features and labels and finally splitting the test and train data.


from sklearn.model_selection import train_test_split
X = cars_data[['year','mileage', 'model_SE', 'model_SEL', 'model_SES',
       'color_Black', 'color_Blue', 'color_Gold', 'color_Gray', 'color_Green',
       'color_Red', 'color_Silver', 'color_White', 'color_Yellow',
       'transmission_AUTO', 'transmission_MANUAL']]
Y = cars_data['price']


X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.20, random_state=1)

#let us scale the data as features are on different scales which might be a problem while modelling
from sklearn import preprocessing
scaler = preprocessing.MinMaxScaler()
# MinMaxScalar has been used here. You can go ahead and use the other scalars available and chcek the effect on the results.
#fitting the transform on test and train separately
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
X_train

array([[0.75      , 0.27928138, 0.        , ..., 0.        , 1.        ,
        0.        ],
       [0.58332825, 0.5233474 , 0.        , ..., 1.        , 0.        ,
        1.        ],
       [0.75      , 0.21721278, 1.        , ..., 0.        , 1.        ,
        0.        ],
       ...,
       [0.83332825, 0.20944397, 0.        , ..., 0.        , 1.        ,
        0.        ],
       [0.33332825, 0.6878496 , 1.        , ..., 0.        , 1.        ,
        0.        ],
       [0.83332825, 0.2253431 , 1.        , ..., 0.        , 1.        ,
        0.        ]], dtype=float32)

In [11]:

# let us now convert the data elements into tensors as we need tensors to be fed into different tensorflow based operations
#X-train and X_test were converted to numpy arrays while transformations while the other two need to be transformed into numpy arrays.
X_train=tf.convert_to_tensor(X_train)
y_train=tf.convert_to_tensor(y_train.values)
X_test=tf.convert_to_tensor(X_test)
y_test=tf.convert_to_tensor(y_test.values)

In [12]:
X_train.shape[1]

16

#### Let us try modelling now. We will use a few concepts covered in the practice exercise shared with the course material.

In [13]:
input_dim = X_train.shape[1]
output_dim = 1
learning_rate = 0.01

# Let us initialize the weights and bias variables. 
weights = tf.Variable(tf.zeros(shape=(input_dim, output_dim), dtype= tf.float32))
bias = tf.Variable(tf.ones(shape=(output_dim,), dtype= tf.float32))

def predict(features):
  return tf.matmul(features, weights) + bias # note that the matmul is matrix multiplication and is needed for calculating predictions

def compute_loss(y_true, predictions):
  return tf.reduce_mean(tf.square(y_true - predictions)) # mean square error

# Let us now define a function to train the model. We will call the other functions in function definition.
def train(x, y):
  with tf.GradientTape() as tape:
    predictions = predict(x)
    loss = compute_loss(y, predictions)
    dloss_dw, dloss_db = tape.gradient(loss, [weights, bias]) #note that we can pass lists as well here.
  weights.assign_sub(learning_rate * dloss_dw)
  bias.assign_sub(learning_rate * dloss_db)
  return loss

#### Let us now, call the train function with 50 epochs

In [14]:

for epoch in range(50):
  loss = train(X_train, y_train)
  print('Epoch %d: Loss = %.4f' % (epoch, float(loss)))


print('Final Weights after 50 epochs:')
print('###############################################################################')
print(weights)

print('Final Bias after 50 epochs:')
print('###############################################################################')
print(bias)

Epoch 0: Loss = 176731120.0000
Epoch 1: Loss = 157933824.0000
Epoch 2: Loss = 141282496.0000
Epoch 3: Loss = 126532160.0000
Epoch 4: Loss = 113465608.0000
Epoch 5: Loss = 101890464.0000
Epoch 6: Loss = 91636368.0000
Epoch 7: Loss = 82552456.0000
Epoch 8: Loss = 74505024.0000
Epoch 9: Loss = 67375672.0000
Epoch 10: Loss = 61059540.0000
Epoch 11: Loss = 55463724.0000
Epoch 12: Loss = 50505948.0000
Epoch 13: Loss = 46113352.0000
Epoch 14: Loss = 42221360.0000
Epoch 15: Loss = 38772792.0000
Epoch 16: Loss = 35717000.0000
Epoch 17: Loss = 33009130.0000
Epoch 18: Loss = 30609446.0000
Epoch 19: Loss = 28482742.0000
Epoch 20: Loss = 26597858.0000
Epoch 21: Loss = 24927154.0000
Epoch 22: Loss = 23446196.0000
Epoch 23: Loss = 22133294.0000
Epoch 24: Loss = 20969272.0000
Epoch 25: Loss = 19937130.0000
Epoch 26: Loss = 19021806.0000
Epoch 27: Loss = 18209964.0000
Epoch 28: Loss = 17489794.0000
Epoch 29: Loss = 16850832.0000
Epoch 30: Loss = 16283805.0000
Epoch 31: Loss = 15780510.0000
Epoch 32: Lo

#### Let us now test our model on the test data and predict on the test data.

In [15]:
test_predictions = tf.matmul(X_test, weights) + bias
print(compute_loss(y_test, test_predictions))

tf.Tensor(8548196.0, shape=(), dtype=float32)


We learnt creating a very simple linear regression model on cars data and predicted prices. 

Though, we could have done an extensive EDA and further improved the model but we have focused on tensorflow and its operations.

<i>Happy Learning!</i>